<a href="https://colab.research.google.com/github/kurtsenol/machine-learning/blob/master/TopicModeling/Methodology/prodigy_model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
!pip install -U pip setuptools wheel
!pip install spacy==3.2.4
!python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import spacy
import os
import warnings
import pickle 
warnings.simplefilter("ignore")

In [3]:
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

pyo.init_notebook_mode(connected=True)

In [4]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
df_m = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/df_method_filtered.pkl")
df_m

,TI,QUANTITATIVE,Method,JO,AU,PY,DI,urls,filename,methodology
0,Barcelona's peer-to-peer tourist accommodation...,1,secondary data,IJCHM,"Benitez-Aurioles, B",2019,10.1108/IJCHM-01-2019-0090,https://www.emerald.com/insight/content/doi/10...,IJCHM_1.txt,Data\nThere are no official statistics on the ...
1,Examining work factors after Malaysia's minimu...,1,survey,IJCHM,"Ahmat, NHC; Arendt, SW; Russell, DW",2019,10.1108/IJCHM-10-2018-0827,https://www.emerald.com/insight/content/doi/10...,IJCHM_3.txt,Methods\nParticipants\nEmployees from operatio...
2,Seeking reward or avoiding risk from restauran...,1,experimental,IJCHM,"Kim, EL; Tanford, S",2019,10.1108/IJCHM-03-2018-0235,https://www.emerald.com/insight/content/doi/10...,IJCHM_4.txt,Method\nDesign\nParticipants made dining decis...
3,Prices of short-stay accommodation: time serie...,1,secondary data,IJCHM,"Gricar, S; Bojnec, S",2019,10.1108/IJCHM-01-2019-0091,https://www.emerald.com/insight/content/doi/10...,IJCHM_5.txt,Empirical setting\nIn terms of relevant econom...
4,How do online reviewers' cultural traits and p...,1,content analysis,IJCHM,"Mariani, M; Predvoditeleva, M",2019,10.1108/IJCHM-11-2018-0927,https://www.emerald.com/insight/content/doi/10...,IJCHM_7.txt,"3. Research setting, design and methodology\n3..."
...,...,...,...,...,...,...,...,...,...,...
2770,"Attitudes, Perceptions, and Responses of Purch...",1.0,survey,JHMM,"Ong, BS",2015,10.1080/19368623.2014.901201,https://www.tandfonline.com/doi/full/10.1080/1...,JHMM_223.txt,\nMETHODOLOGY\nGiven that daily deals are Inte...
2771,Exploring the Antecedents of Hotel Customer Lo...,1.0,survey,JHMM,"de Leaniz, PMG; Rodriguez, IRD",2015,10.1080/19368623.2014.891961,https://www.tandfonline.com/doi/full/10.1080/1...,JHMM_224.txt,\nMETHOD OF ANALYSIS\nData Collection and Samp...
2772,Casual Dining on the French Riviera: Examining...,1.0,survey,JHMM,"Prayag, G; Khoo-Lattimore, C; Sitruk, J",2015,10.1080/19368623.2014.859114,https://www.tandfonline.com/doi/full/10.1080/1...,JHMM_225.txt,\nMETHODOLOGY\nThe Study Site\nThe French Rivi...
2773,The Influence of DMO Advertising on Specific D...,1.0,survey,JHMM,"Morosan, C",2015,10.1080/19368623.2014.891962,https://www.tandfonline.com/doi/full/10.1080/1...,JHMM_226.txt,\nMETHODOLOGY\nInstrument Development\nThe res...


In [5]:
with open('/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/prodigy_results_last.pkl', 'rb') as f:
  res_last_1 = pickle.load(f)

In [6]:
with open('/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/prodigy_2_results_last.pkl', 'rb') as f:
  res_last_2 = pickle.load(f)

In [7]:
df_last_1 = pd.DataFrame(res_last_1)
df_last_1.columns = ["Document", "Methodology", "Entities"]
df_last_1["Entity_Count"] = df_last_1["Entities"].apply(len)
df_last_1["Unique_Entity_Count"] = df_last_1["Entities"].apply(set).apply(len)

In [12]:
df_last_2 = pd.DataFrame(res_last_2)
df_last_2.columns = ["Document", "Methodology", "Entities"]
df_last_2["Entity_Count"] = df_last_2["Entities"].apply(len)
df_last_2["Unique_Entity_Count"] = df_last_2["Entities"].apply(set).apply(len)

In [10]:
configure_plotly_browser_state()

fig = px.box(df_last_1, y = "Entity_Count", x = "Methodology", hover_data=["Document"])
fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title_text=f"First Model Entity Counts Per Methodology",

)
fig.show()

In [13]:
configure_plotly_browser_state()

fig = px.box(df_last_2, y = "Entity_Count", x = "Methodology", hover_data=["Document"])
fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title_text=f"Second Model Entity Counts Per Methodology",

)
fig.show()

In [14]:
configure_plotly_browser_state()

fig = px.box(df_last_1, y = "Unique_Entity_Count", x = "Methodology", hover_data=["Document"])
fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title_text=f"First Model Unique Entity Counts Per Methodology",

)
fig.show()

In [15]:
configure_plotly_browser_state()

fig = px.box(df_last_2, y = "Unique_Entity_Count", x = "Methodology", hover_data=["Document"])
fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title_text=f"First Model Unique Entity Counts Per Methodology",

)
fig.show()

In [18]:
print("First Model Paper Counts per Entiy Counts")
for i in range(1,11):
  print("# of unique entities", i -1 , "or less:", df_last_1[df_last_1["Unique_Entity_Count"]<i].shape[0], "papers")

First Model Paper Counts per Entiy Counts
# of unique entities 0 or less: 43 papers
# of unique entities 1 or less: 180 papers
# of unique entities 2 or less: 395 papers
# of unique entities 3 or less: 667 papers
# of unique entities 4 or less: 964 papers
# of unique entities 5 or less: 1272 papers
# of unique entities 6 or less: 1567 papers
# of unique entities 7 or less: 1811 papers
# of unique entities 8 or less: 2035 papers
# of unique entities 9 or less: 2225 papers


In [19]:
print("Second Model Paper Counts per Entiy Counts")
for i in range(1,11):
  print("# of unique entities", i -1 , "or less:", df_last_2[df_last_2["Unique_Entity_Count"]<i].shape[0], "papers")

Second Model Paper Counts per Entiy Counts
# of unique entities 0 or less: 7 papers
# of unique entities 1 or less: 40 papers
# of unique entities 2 or less: 93 papers
# of unique entities 3 or less: 169 papers
# of unique entities 4 or less: 287 papers
# of unique entities 5 or less: 408 papers
# of unique entities 6 or less: 585 papers
# of unique entities 7 or less: 754 papers
# of unique entities 8 or less: 924 papers
# of unique entities 9 or less: 1079 papers
